### 🏎️ Formula 1 서킷의 성향 분석

**데이터 분석 방법론 (Methodology)**
- 피처 엔지니어링 (Feature Engineering): 텔레메트리 데이터에서 **최고 속도(Max)**와 직선 주행을 배제한 **순수 코너링 속도(하위 15% 평균)**를 독립 변수로 추출.
- 모델링 (Modeling): 2018~2025년 데이터를 다중 선형 회귀(Multiple Linear Regression) 모델에 학습시켜 랩타임($Y$) 결정 요인을 규명.
- 영향력 산출 (Importance): 단위 차이를 제거하기 위해 표준화(Standard Scaling) 적용 후, 회귀 계수의 비중을 산출하여 서킷 성향을 정량적으로 분류.

In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go

# === [설정] ===
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "f1_analytics_db"

YEARS = range(2018, 2026)

# 주요 그랑프리 한글 매핑 (Event Name 기준)
CIRCUIT_MAP = {
    "Bahrain Grand Prix": "바레인",
    "Saudi Arabian Grand Prix": "제다 (사우디)",
    "Australian Grand Prix": "멜버른 (호주)",
    "Japanese Grand Prix": "스즈카 (일본)",
    "Chinese Grand Prix": "상하이 (중국)",
    "Miami Grand Prix": "마이애미 (미국)",
    "Emilia Romagna Grand Prix": "이몰라 (이탈리아)",
    "Monaco Grand Prix": "모나코",
    "Canadian Grand Prix": "몬트리올 (캐나다)",
    "Spanish Grand Prix": "바르셀로나 (스페인)",
    "Austrian Grand Prix": "레드불링 (오스트리아)",
    "British Grand Prix": "실버스톤 (영국)",
    "Hungarian Grand Prix": "헝가로링 (헝가리)",
    "Belgian Grand Prix": "스파 (벨기에)",
    "Dutch Grand Prix": "잔트보르트 (네덜란드)",
    "Italian Grand Prix": "몬자 (이탈리아)",
    "Azerbaijan Grand Prix": "바쿠 (아제르바이잔)",
    "Singapore Grand Prix": "싱가포르",
    "United States Grand Prix": "오스틴 (미국)",
    "Mexico City Grand Prix": "멕시코시티",
    "São Paulo Grand Prix": "인터라고스 (브라질)",
    "Las Vegas Grand Prix": "라스베가스",
    "Qatar Grand Prix": "루사일 (카타르)",
    "Abu Dhabi Grand Prix": "야스마리나 (아부다비)",
    "French Grand Prix": "폴리카르 (프랑스)", # 과거 경기
    "Russian Grand Prix": "소치 (러시아)",     # 과거 경기
    "Portuguese Grand Prix": "포르티망 (포르투갈)",
    "Turkish Grand Prix": "이스탄불 (터키)",
    "Eifel Grand Prix": "뉘르부르크링 (독일)",
    "Sakhir Grand Prix": "사키르 (바레인 숏)",
    "Styrian Grand Prix": "스티리안 (오스트리아)",
    "70th Anniversary Grand Prix": "실버스톤 70주년"
}

1. 모든 서킷의 퀄리파잉(예선) 데이터를 순회하며, 폴포지션(1위) 랩의 **최고속도**와 **코너링 속도(하위 15% 평균)**를 추출하여 데이터셋을 생성

In [2]:
def get_all_circuits_data():
    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]

    dataset = []

    # 1. DB에 존재하는 모든 'Circuit' (Event Name) 종류 조회
    all_events = db.races_meta.distinct("Circuit")
    print(f"🌍 총 {len(all_events)}개 서킷 데이터 스캔 시작...")

    for event_name in all_events:
        # 한글 이름 매핑 (없으면 영문 그대로)
        circuit_kor = CIRCUIT_MAP.get(event_name, event_name)

        # 데이터 수집 (연도별 루프)
        for year in YEARS:
            # 메타데이터 조회
            race_meta = db.races_meta.find_one({
                "Year": year,
                "SessionType": "Qualifying",
                "Circuit": event_name
            })
            if not race_meta: continue

            round_num = race_meta['Round']

            # 드라이버 매핑 (Driver ID Issue 해결용)
            driver_results = list(db.results.find(
                {"Year": year, "Round": round_num, "SessionType": "Qualifying"},
                {"Driver": 1, "DriverNumber": 1, "_id": 0}
            ))
            driver_map = {d['Driver']: str(d['DriverNumber']) for d in driver_results}

            # Fastest Lap 조회
            fastest_lap = db.laps.find_one(
                {
                    "Year": year, "Round": round_num,
                    "SessionType": "Qualifying", "IsAccurate": True
                },
                sort=[("LapTime_Sec", 1)]
            )
            if not fastest_lap: continue

            driver_abbr = fastest_lap['Driver']
            if driver_abbr not in driver_map: continue
            driver_num = driver_map[driver_abbr]

            # 텔레메트리 조회
            telemetry_cursor = db.telemetry.find(
                {
                    "Year": year, "Round": round_num,
                    "SessionType": "Qualifying", "Driver": driver_num
                },
                {"Speed": 1, "_id": 0}
            )

            speeds = [d['Speed'] for d in telemetry_cursor if d['Speed'] is not None]

            if len(speeds) > 100:
                top_speed = np.max(speeds)
                valid_speeds = [s for s in speeds if s > 60]
                # 코너링 속도: 하위 15%
                corner_speed = np.percentile(valid_speeds, 15) if valid_speeds else np.mean(speeds)

                dataset.append({
                    "Circuit": circuit_kor, # 한글 이름 저장
                    "OriginalName": event_name,
                    "Year": year,
                    "PoleTime": fastest_lap['LapTime_Sec'],
                    "TopSpeed": top_speed,
                    "CornerSpeed": corner_speed
                })

    print(f"✅ 총 {len(dataset)}개의 랩 데이터 확보.")
    return pd.DataFrame(dataset)

2. 각 서킷별로 다중 선형 회귀 분석을 수행하여 랩타임 결정 요인의 **상대적 중요도(%)** 산출 및 파워 트랙 순서대로 정렬

In [3]:
def analyze_and_sort(df):
    results = []
    print("\n🤖 서킷별 회귀 분석 진행 중...")

    for circuit in df['Circuit'].unique():
        subset = df[df['Circuit'] == circuit].copy()

        # 데이터 포인트 3개 미만이면 분석 제외 (신뢰도 문제)
        if len(subset) < 3:
            continue

        X = subset[['TopSpeed', 'CornerSpeed']]
        y = subset['PoleTime']

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        model = LinearRegression()
        model.fit(X_scaled, y)

        imp_top = abs(model.coef_[0])
        imp_corner = abs(model.coef_[1])
        total = imp_top + imp_corner + 1e-9 # 0나눗셈 방지

        results.append({
            "Circuit": circuit,
            "TopSpeed_Pct": (imp_top / total) * 100,
            "CornerSpeed_Pct": (imp_corner / total) * 100,
            "DataCount": len(subset)
        })

    res_df = pd.DataFrame(results)

    # Top Speed 비중이 높은 순서대로 정렬 (Power Track -> Technical Track)
    res_df = res_df.sort_values(by="TopSpeed_Pct", ascending=True)
    return res_df

3. Plotly 막대 그래프 시각화

In [4]:
def visualize_all_circuits(res_df):
    print("🎨 전체 서킷 분석 차트 생성 중...")

    # 서킷 개수에 따라 높이 동적 조절 (서킷당 40px)
    chart_height = max(600, len(res_df) * 40)

    fig = go.Figure()

    # 최고 속도 (Top Speed)
    fig.add_trace(go.Bar(
        y=res_df['Circuit'],
        x=res_df['TopSpeed_Pct'],
        name='최고 속도 (Top Speed)',
        orientation='h',
        marker=dict(color='#00D2BE'), # Cyan
        hovertemplate='%{y}<br>최고 속도 중요도: %{x:.1f}%<extra></extra>'
    ))

    # 코너링 속도 (Corner Speed)
    fig.add_trace(go.Bar(
        y=res_df['Circuit'],
        x=res_df['CornerSpeed_Pct'],
        name='코너링 속도 (Corner Speed)',
        orientation='h',
        marker=dict(color='#DC0000'), # Red
        hovertemplate='%{y}<br>코너링 속도 중요도: %{x:.1f}%<extra></extra>'
    ))

    fig.update_layout(
        title="🏎️ F1 전 서킷 성향 분석: 파워 트랙 vs 테크니컬 트랙",
        xaxis_title="상대적 중요도 (%)",
        yaxis_title="",
        barmode='stack',
        template="plotly_dark",
        height=chart_height, # 동적 높이 적용
        margin=dict(l=150, r=20, t=60, b=40), # 왼쪽 여백 확보 (긴 이름)
        font=dict(
            family="Malgun Gothic, AppleGothic, sans-serif",
            size=12,
            color="white"
        ),
        legend=dict(orientation="h", yanchor="bottom", y=1.01, xanchor="right", x=1)
    )

    fig.update_xaxes(range=[0, 100], ticksuffix="%")
    fig.show()

In [5]:
if __name__ == "__main__":
    df = get_all_circuits_data()

    if not df.empty:
        res = analyze_and_sort(df)
        if not res.empty:
            visualize_all_circuits(res)
        else:
            print("❌ 분석 실패: 데이터 부족.")
    else:
        print("❌ 데이터 없음.")

🌍 총 36개 서킷 데이터 스캔 시작...
✅ 총 171개의 랩 데이터 확보.

🤖 서킷별 회귀 분석 진행 중...
🎨 전체 서킷 분석 차트 생성 중...
✅ HTML 저장 완료: 1_Circuit_Importance.html
✅ 이미지 저장 완료: 1_Circuit_Importance.png


**2018 ~ 2025년 퀄리파잉(예선) 데이터를 기반으로 PoleTime ~ TopSpeed + CornerSpeed 회귀 모델을 수립하여 변수 중요도 추출**
- 청록색이 길수록 : 코너에서 손해를 보더라도 직선에서 무조건 빨라야 유리하다 (Power Sensitivity High)
- 빨간색이 길수록 : 직선에서 좀 느려도 코너를 빠르게 도는것이 유리하다 (Downforce Sensitivity High)

* **직선 가속이 승부를 가르는 서킷**
    * 서킷 : 마이애미, 루사일, 레드불링, 실버스톤 등
    * 분석 : 이 서킷들은 긴 직선 구간과 고속 코너가 결합되어, 공기 저항(Drag)을 줄이는 것이 랩타임에 가장 큰 영향을 미칩니다.
    * 인사이트:
        * 마이애미/카타르 : DRS 존이 길고 노폭이 넓어 엔진 파워와 최고 속도의 영향력이 절대적인 서킷
        * 실버스톤 : 고속 코너가 많지만 코너를 '감속'해서 도는 것이 아닌 '풀 악셀'에 가깝게 돌기 때문에 데이터 상으로는 '속도'의 영역으로 해석.
<br/>

* **밸런스의 서킷**
    * 서킷 : 바레인, 이몰라, 스파, 바르셀로나 등
    * 분석 : 색의 분포가 5:5 또는 4:6 정도로 균형 잡혀 있습니다.
    * 인사이트:
        * 이 서킷들은 엔진 성능만 좋아서도, 다운포스만 높아서도 안 됩니다. 차량의 종합적인 밸런스(Setup Compromise)를 가장 잘 맞춘 팀이 폴 포지션(예선 1등)을 차지한다는 것을 의미.
<br/>

* **코너링의 테크니컬 서킷**
    * 서킷 : 헝가로링, 스즈카, 모나코 등
    * 분석 : 코너링 중요도가 압도적인 서킷들입니다.
    * 인사이트:
        * 헝가로링과 스즈카는 랩 최고 속도가 300km/h를 넘더라도, 결국 복잡한 섹터에서의 코너링 속도가 랩타임을 결정짓습니다. 엔지니어들은 이곳에서 항력을 희생하더라도 최대 다운포스(Max Downforce) 세팅을 가져와야 함을 의미합니다.

- 의외의 결과 - 몬자와 바쿠
    - 몬자는 '속도의 사원(Temple of Speed)'이라 불리며 F1에서 가장 빠른 서킷입니다. 하지만 데이터는 **"코너링 속도가 70% 이상 중요하다"**고 말합니다.
    - 해석
        - 변별력의 차이 : 몬자에서는 모든 팀이 가장 얇은 윙(Low Downforce)을 씁니다. 즉, 1등과 10등의 직선 속도 차이가 거의 없습니다. (변별력이 낮음) 바쿠 또한 마찬가지입니다.
        - 승부처의 이동 : 다들 직선에서 빠르기 때문에, 결국 랩타임의 0.01초 차이를 만드는 곳은 시케인(Chicane)과 레스모(Lesmo) 같은 저속 코너입니다.
        - **결론 : 몬자에서 이기려면 직선은 기본이고, 남들보다 시케인(어려운 코너)을 과감하게 공략해야 합니다. 바쿠 또한 극도로 좁은 섹터 2의 코너링 공략이 매우 중요합니다.**